# Import Needed Libraries

In [11]:
from statistics import mean,stdev
from termcolor import colored
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [12]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)    
    return first_nodes

In [13]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)   
    return bottom_nodes

In [14]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# LT Simulations for all Networks

In [16]:
networks = next(walk(cleanNetworks), (None, None, []))[2]

for network in networks:
    counter = 0
    g = nx.read_gml(f'{cleanNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]
    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))
    
    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraph(max)
    #print(network)
    
    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])
        
    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/SyntheticNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
        #print(dictionary)
    
    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_average_each_simulation'] = dict()
        globals()[f'{centrality}_standard_each_simulation'] = dict()
        for fraction in fractions: 
            globals()[f'{centrality}_average_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standard_each_simulation'][fraction] = 0
    
    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')
    
    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')
    
    #thresholds
    randomThresholds = np.random.uniform(low=0, high=1, size=len(g.nodes()))
    fixedThresholds = [0.4,0.7]
    
    #Run diffusion model on centralities
    for counter in range(3): 
        for centrality in colors.keys(): 
            print(colored(f'Counter: {counter}, Centrality: {centrality}', 'green',attrs=['bold']))
            for fraction in fractions:

                globals()[f'l_{centrality}'] = list()
                fraction_infected = round(fraction*network_size)

                if(centrality=="modv_neg"):
                    infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                else:
                    infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)

                
                
                if(counter==0 or counter==1):
                    model = ep.ThresholdModel(g)
                    config = mc.Configuration()
                    config.add_model_initial_configuration('Infected', infected_nodes)
                    threshold = fixedThresholds[counter]

                    for i in g.nodes():
                        config.add_node_configuration("threshold", i, threshold)

                    model.set_initial_status(config)
              
                    x = 1
                    iterations = model.iteration_bunch(1)
                    iterations = iterations + model.iteration_bunch(1)
                    while(iterations[x-1]['node_count']!=iterations[x]['node_count']):
                        iterations = iterations + model.iteration_bunch(1)
                        x = x+1

                    finalInfectedInOneIteration = iterations[x]['node_count'][1]
                
                    globals()[f'l_{centrality}'].append(finalInfectedInOneIteration)
                    globals()[f'{centrality}_average_each_simulation'][fraction] = finalInfectedInOneIteration
                    #print( globals()[f'{centrality}_average_each_simulation'])
                    p = f"./Pickle Variables/{n}/Threshold-{threshold}/"
                    if(path.exists(p)==False):
                        os.mkdir(p)
                    save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
                    print(fraction,":",finalInfectedInOneIteration)

                else:
                    for i in range(repeatedSimulations):
                        model = ep.ThresholdModel(g)
                        config = mc.Configuration()
                        config.add_model_initial_configuration('Infected', infected_nodes)
                        random.shuffle(randomThresholds)
                        for i,threshold in zip(g.nodes(),randomThresholds):
                            config.add_node_configuration("threshold", i, threshold)

                        model.set_initial_status(config)
                        x = 1
                        iterations = model.iteration_bunch(1)
                        iterations = iterations + model.iteration_bunch(1)
                        while(iterations[x-1]['node_count']!=iterations[x]['node_count']):
                            iterations = iterations + model.iteration_bunch(1)
                            x = x+1

                        finalInfectedInOneIteration = iterations[x]['node_count'][1]
                        
                        globals()[f'l_{centrality}'].append(finalInfectedInOneIteration)

                    globals()[f'{centrality}_average_each_simulation'][fraction] = mean(globals()[f'l_{centrality}'])
                    globals()[f'{centrality}_standard_each_simulation'][fraction] = stdev(globals()[f'l_{centrality}'])
                    print(fraction,":", mean(globals()[f'l_{centrality}']))
                    p = f"./Pickle Variables/{n}/UniformRandomThresholds/"
                    if(path.exists(p)==False):
                        os.mkdir(p)
                    save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
                    save_obj(globals()[f'{centrality}_standard_each_simulation'], p ,f'{centrality}_standard_each_simulation')

Simulation currently on network networkovermu035.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10417
Average degree:   8.3336...
Counter: 0, Centrality: modv_pos
0.01 : 25
0.02 : 51
0.03 : 79
0.04 : 117
0.05 : 171
0.06 : 228
0.07 : 280
0.08 : 334
0.09 : 380
0.1 : 427
0.11 : 464
0.12 : 514
0.13 : 566
0.14 : 619
0.15 : 726
0.16 : 2500
0.17 : 2500
0.18 : 2500
0.19 : 2500
0.2 : 2500
0.21 : 2500
0.22 : 2500
0.23 : 2500
0.24 : 2500
0.25 : 2500
0.26 : 2500
0.27 : 2500
0.28 : 2500
0.29 : 2500
0.3 : 2500
0.31 : 2500
0.32 : 2500
0.33 : 2500
0.34 : 2500
0.35 : 2500
0.36 : 2500
0.37 : 2500
0.38 : 2500
0.39 : 2500
0.4 : 2500
0.41 : 2500
0.42 : 2500
0.43 : 2500
0.44 : 2500
0.45 : 2500
0.46 : 2500
0.47 : 2500
0.48 : 2500
0.49 : 2500
0.5 : 2500
Counter: 0, Centrality: com
0.01 : 46
0.02 : 90
0.03 : 131
0.04 : 189
0.05 : 276
0.06 : 371
0.07 : 449
0.08 : 668
0.09 : 795
0.1 : 1050
0.11 : 2500
0.12 : 2500
0.13 : 2500
0.14 : 2500
0.15 : 2500
0.16 : 2500
0.17 : 2500
0.18 : 2500
0.19 : 2500

0.47 : 1628
0.48 : 1668
0.49 : 1694
0.5 : 1730
Counter: 1, Centrality: cbm
0.01 : 25
0.02 : 50
0.03 : 76
0.04 : 102
0.05 : 130
0.06 : 162
0.07 : 193
0.08 : 225
0.09 : 268
0.1 : 311
0.11 : 349
0.12 : 386
0.13 : 427
0.14 : 470
0.15 : 526
0.16 : 573
0.17 : 632
0.18 : 684
0.19 : 723
0.2 : 769
0.21 : 813
0.22 : 849
0.23 : 875
0.24 : 921
0.25 : 955
0.26 : 991
0.27 : 1053
0.28 : 1105
0.29 : 1162
0.3 : 1219
0.31 : 1273
0.32 : 1297
0.33 : 1339
0.34 : 1376
0.35 : 1409
0.36 : 1449
0.37 : 1466
0.38 : 1485
0.39 : 1505
0.4 : 1527
0.41 : 1561
0.42 : 1598
0.43 : 1628
0.44 : 1651
0.45 : 1676
0.46 : 1704
0.47 : 1734
0.48 : 1773
0.49 : 1792
0.5 : 1831
Counter: 1, Centrality: chb
0.01 : 25
0.02 : 51
0.03 : 75
0.04 : 101
0.05 : 128
0.06 : 155
0.07 : 183
0.08 : 208
0.09 : 236
0.1 : 255
0.11 : 282
0.12 : 309
0.13 : 346
0.14 : 367
0.15 : 395
0.16 : 412
0.17 : 439
0.18 : 468
0.19 : 501
0.2 : 528
0.21 : 565
0.22 : 596
0.23 : 623
0.24 : 659
0.25 : 690
0.26 : 720
0.27 : 763
0.28 : 796
0.29 : 836
0.3 : 866
0.31 : 

0.26 : 1609.54
0.27 : 1637.43
0.28 : 1677.44
0.29 : 1716.42
0.3 : 1732.69
0.31 : 1747.1
0.32 : 1768.09
0.33 : 1799.25
0.34 : 1832.63
0.35 : 1860.29
0.36 : 1884.1
0.37 : 1908.89
0.38 : 1923.3
0.39 : 1952.82
0.4 : 1968.58
0.41 : 1995.26
0.42 : 2018.41
0.43 : 2033.25
0.44 : 2044.05
0.45 : 2052.62
0.46 : 2072.3
0.47 : 2083.4
0.48 : 2111.54
0.49 : 2124.05
0.5 : 2131.81
Counter: 2, Centrality: pc
0.01 : 97.51
0.02 : 198.13
0.03 : 285.23
0.04 : 402.6
0.05 : 578.38
0.06 : 696.98
0.07 : 771.54
0.08 : 866.41
0.09 : 919.57
0.1 : 1018.63
0.11 : 1092.4
0.12 : 1182.95
0.13 : 1261.11
0.14 : 1348.51
0.15 : 1457.3
0.16 : 1542.34
0.17 : 1606.61
0.18 : 1641.73
0.19 : 1691.03
0.2 : 1724.35
0.21 : 1754.04
0.22 : 1798.29
0.23 : 1822.69
0.24 : 1867.72
0.25 : 1902.31
0.26 : 1929.8
0.27 : 1960.78
0.28 : 1991.94
0.29 : 2010.2
0.3 : 2041.55
0.31 : 2063.99
0.32 : 2092.11
0.33 : 2111.64
0.34 : 2125.06
0.35 : 2141.58
0.36 : 2160.43
0.37 : 2175.35
0.38 : 2176.78
0.39 : 2199.59
0.4 : 2206.23
0.41 : 2216.11
0.42 : 223

0.32 : 1836
0.33 : 1961
0.34 : 1969
0.35 : 2043
0.36 : 2043
0.37 : 2120
0.38 : 2127
0.39 : 2127
0.4 : 2127
0.41 : 2143
0.42 : 2190
0.43 : 2270
0.44 : 2286
0.45 : 2295
0.46 : 2313
0.47 : 2313
0.48 : 2335
0.49 : 2353
0.5 : 2379
Counter: 0, Centrality: pc
0.01 : 26
0.02 : 52
0.03 : 80
0.04 : 111
0.05 : 152
0.06 : 217
0.07 : 270
0.08 : 320
0.09 : 390
0.1 : 450
0.11 : 527
0.12 : 565
0.13 : 618
0.14 : 641
0.15 : 696
0.16 : 779
0.17 : 840
0.18 : 873
0.19 : 907
0.2 : 951
0.21 : 1009
0.22 : 1045
0.23 : 2112
0.24 : 2190
0.25 : 2230
0.26 : 2257
0.27 : 2266
0.28 : 2302
0.29 : 2317
0.3 : 2326
0.31 : 2333
0.32 : 2344
0.33 : 2356
0.34 : 2366
0.35 : 2383
0.36 : 2388
0.37 : 2402
0.38 : 2416
0.39 : 2421
0.4 : 2435
0.41 : 2435
0.42 : 2450
0.43 : 2461
0.44 : 2461
0.45 : 2479
0.46 : 2479
0.47 : 2479
0.48 : 2479
0.49 : 2479
0.5 : 2479
Counter: 0, Centrality: kshell
0.01 : 37
0.02 : 115
0.03 : 262
0.04 : 262
0.05 : 290
0.06 : 344
0.07 : 578
0.08 : 1315
0.09 : 1341
0.1 : 1395
0.11 : 1442
0.12 : 1521
0.13 : 16

0.03 : 78
0.04 : 107
0.05 : 135
0.06 : 167
0.07 : 203
0.08 : 239
0.09 : 271
0.1 : 307
0.11 : 332
0.12 : 359
0.13 : 388
0.14 : 415
0.15 : 444
0.16 : 468
0.17 : 494
0.18 : 511
0.19 : 536
0.2 : 570
0.21 : 603
0.22 : 630
0.23 : 653
0.24 : 676
0.25 : 708
0.26 : 737
0.27 : 765
0.28 : 794
0.29 : 817
0.3 : 854
0.31 : 882
0.32 : 911
0.33 : 940
0.34 : 961
0.35 : 989
0.36 : 1016
0.37 : 1042
0.38 : 1076
0.39 : 1115
0.4 : 1151
0.41 : 1191
0.42 : 1225
0.43 : 1254
0.44 : 1285
0.45 : 1313
0.46 : 1337
0.47 : 1365
0.48 : 1400
0.49 : 1425
0.5 : 1449
Counter: 1, Centrality: modv_neg
0.01 : 25
0.02 : 51
0.03 : 76
0.04 : 101
0.05 : 126
0.06 : 156
0.07 : 188
0.08 : 219
0.09 : 244
0.1 : 269
0.11 : 295
0.12 : 322
0.13 : 360
0.14 : 394
0.15 : 423
0.16 : 454
0.17 : 483
0.18 : 517
0.19 : 551
0.2 : 575
0.21 : 598
0.22 : 627
0.23 : 655
0.24 : 694
0.25 : 723
0.26 : 753
0.27 : 784
0.28 : 810
0.29 : 836
0.3 : 864
0.31 : 896
0.32 : 923
0.33 : 953
0.34 : 976
0.35 : 989
0.36 : 1007
0.37 : 1035
0.38 : 1053
0.39 : 1082
0.4

...

0.42 : 1124
0.43 : 1156
0.44 : 1191
0.45 : 1211
0.46 : 1240
0.47 : 1271
0.48 : 1315
0.49 : 1340
0.5 : 1369
Counter: 1, Centrality: pc
0.01 : 25
0.02 : 50
0.03 : 75
0.04 : 100
0.05 : 125
0.06 : 150
0.07 : 176
0.08 : 200
0.09 : 225
0.1 : 250
0.11 : 275
0.12 : 300
0.13 : 325
0.14 : 350
0.15 : 379
0.16 : 404
0.17 : 429
0.18 : 456
0.19 : 485
0.2 : 510
0.21 : 535
0.22 : 561
0.23 : 591
0.24 : 619
0.25 : 647
0.26 : 673
0.27 : 703
0.28 : 734
0.29 : 760
0.3 : 783
0.31 : 811
0.32 : 841
0.33 : 868
0.34 : 896
0.35 : 926
0.36 : 960
0.37 : 996
0.38 : 1043
0.39 : 1088
0.4 : 1136
0.41 : 1194
0.42 : 1258
0.43 : 1325
0.44 : 1395
0.45 : 1476
0.46 : 1637
0.47 : 1737
0.48 : 1768
0.49 : 1815
0.5 : 1842
Counter: 1, Centrality: kshell
0.01 : 27
0.02 : 54
0.03 : 84
0.04 : 110
0.05 : 142
0.06 : 174
0.07 : 207
0.08 : 245
0.09 : 283
0.1 : 319
0.11 : 372
0.12 : 416
0.13 : 447
0.14 : 483
0.15 : 518
0.16 : 554
0.17 : 598
0.18 : 654
0.19 : 698
0.2 : 738
0.21 : 778
0.22 : 800
0.23 : 832
0.24 : 839
0.25 : 877
0.26 : 918

0.01 : 229.67
0.02 : 325.33
0.03 : 478.6
0.04 : 690.89
0.05 : 795.61
0.06 : 894.35
0.07 : 952.5
0.08 : 1024.73
0.09 : 1076.73
0.1 : 1127.72
0.11 : 1137.2
0.12 : 1159.17
0.13 : 1206.4
0.14 : 1264.86
0.15 : 1337.74
0.16 : 1408.61
0.17 : 1440.95
0.18 : 1479.6
0.19 : 1506.94
0.2 : 1529.69
0.21 : 1557.03
0.22 : 1562.32
0.23 : 1585.54
0.24 : 1595.94
0.25 : 1626.97
0.26 : 1668.08
0.27 : 1716.77
0.28 : 1762.09
0.29 : 1794.76
0.3 : 1842.25
0.31 : 1868.03
0.32 : 1892.4
0.33 : 1925.81
0.34 : 1944.06
0.35 : 1981.22
0.36 : 2008.31
0.37 : 2025.36
0.38 : 2032.25
0.39 : 2047.11
0.4 : 2058.21
0.41 : 2066.96
0.42 : 2068.25
0.43 : 2075.14
0.44 : 2088.33
0.45 : 2099.65
0.46 : 2119.23
0.47 : 2134.95
0.48 : 2155.18
0.49 : 2167.4
0.5 : 2180.66
Counter: 2, Centrality: cbc
0.01 : 311.03
0.02 : 481.63
0.03 : 616.08
0.04 : 704.18
0.05 : 767.24
0.06 : 801.88
0.07 : 863.82
0.08 : 901.99
0.09 : 971.33
0.1 : 997.23
0.11 : 1040.28
0.12 : 1084.35
0.13 : 1118.35
0.14 : 1151.28
0.15 : 1220.79
0.16 : 1254.6
0.17 : 1277.5

0.27 : 1372
0.28 : 1398
0.29 : 1404
0.3 : 1412
0.31 : 1415
0.32 : 1435
0.33 : 1435
0.34 : 1443
0.35 : 1444
0.36 : 1444
0.37 : 1444
0.38 : 1484
0.39 : 1554
0.4 : 1583
0.41 : 1619
0.42 : 1660
0.43 : 1692
0.44 : 1726
0.45 : 1792
0.46 : 1830
0.47 : 1866
0.48 : 1928
0.49 : 1982
0.5 : 2022
Counter: 0, Centrality: cbc
0.01 : 172
0.02 : 187
0.03 : 286
0.04 : 303
0.05 : 452
0.06 : 454
0.07 : 459
0.08 : 460
0.09 : 503
0.1 : 507
0.11 : 521
0.12 : 637
0.13 : 646
0.14 : 647
0.15 : 700
0.16 : 733
0.17 : 750
0.18 : 790
0.19 : 807
0.2 : 850
0.21 : 872
0.22 : 909
0.23 : 936
0.24 : 982
0.25 : 1041
0.26 : 1064
0.27 : 1110
0.28 : 1165
0.29 : 1207
0.3 : 1249
0.31 : 1331
0.32 : 1406
0.33 : 1503
0.34 : 1548
0.35 : 1574
0.36 : 1613
0.37 : 1727
0.38 : 1802
0.39 : 1834
0.4 : 1932
0.41 : 1975
0.42 : 2000
0.43 : 2026
0.44 : 2041
0.45 : 2063
0.46 : 2095
0.47 : 2139
0.48 : 2147
0.49 : 2172
0.5 : 2185
Counter: 0, Centrality: modv_neg
0.01 : 172
0.02 : 174
0.03 : 183
0.04 : 290
0.05 : 301
0.06 : 311
0.07 : 410
0.08 :

0.03 : 84
0.04 : 109
0.05 : 134
0.06 : 165
0.07 : 195
0.08 : 222
0.09 : 250
0.1 : 279
0.11 : 305
0.12 : 330
0.13 : 355
0.14 : 382
0.15 : 414
0.16 : 444
0.17 : 477
0.18 : 505
0.19 : 538
0.2 : 569
0.21 : 599
0.22 : 624
0.23 : 652
0.24 : 672
0.25 : 698
0.26 : 723
0.27 : 752
0.28 : 781
0.29 : 807
0.3 : 825
0.31 : 854
0.32 : 899
0.33 : 930
0.34 : 962
0.35 : 991
0.36 : 1042
0.37 : 1078
0.38 : 1114
0.39 : 1159
0.4 : 1195
0.41 : 1242
0.42 : 1286
0.43 : 1315
0.44 : 1366
0.45 : 1414
0.46 : 1463
0.47 : 1522
0.48 : 1593
0.49 : 1647
0.5 : 1723
Counter: 1, Centrality: degree
0.01 : 25
0.02 : 50
0.03 : 76
0.04 : 105
0.05 : 137
0.06 : 168
0.07 : 201
0.08 : 246
0.09 : 286
0.1 : 324
0.11 : 384
0.12 : 427
0.13 : 488
0.14 : 547
0.15 : 599
0.16 : 643
0.17 : 713
0.18 : 769
0.19 : 817
0.2 : 860
0.21 : 908
0.22 : 944
0.23 : 1012
0.24 : 1048
0.25 : 1082
0.26 : 1122
0.27 : 1177
0.28 : 1242
0.29 : 1305
0.3 : 1322
0.31 : 1334
0.32 : 1373
0.33 : 1419
0.34 : 1476
0.35 : 1506
0.36 : 1544
0.37 : 1598
0.38 : 1604
0.39

0.16 : 1568.33
0.17 : 1617.82
0.18 : 1652.79
0.19 : 1697.41
0.2 : 1753.02
0.21 : 1788.44
0.22 : 1823.86
0.23 : 1861.24
0.24 : 1871.65
0.25 : 1897.03
0.26 : 1930.39
0.27 : 1956.04
0.28 : 1992.08
0.29 : 2004.26
0.3 : 2023.75
0.31 : 2046.69
0.32 : 2068.35
0.33 : 2087.02
0.34 : 2103.97
0.35 : 2115.79
0.36 : 2138.56
0.37 : 2167.85
0.38 : 2182.33
0.39 : 2201.94
0.4 : 2218.54
0.41 : 2241.06
0.42 : 2252.52
0.43 : 2264.59
0.44 : 2270.21
0.45 : 2280.44
0.46 : 2296.01
0.47 : 2309.49
0.48 : 2321.22
0.49 : 2331.55
0.5 : 2341.07
Counter: 2, Centrality: degree
0.01 : 420.53
0.02 : 654.32
0.03 : 807.91
0.04 : 938.07
0.05 : 1022.48
0.06 : 1102.23
0.07 : 1179.59
0.08 : 1225.68
0.09 : 1275.2
0.1 : 1337.86
0.11 : 1376.38
0.12 : 1412.71
0.13 : 1458.16
0.14 : 1475.6
0.15 : 1527.8
0.16 : 1558.7
0.17 : 1570.92
0.18 : 1608.9
0.19 : 1658.29
0.2 : 1673.84
0.21 : 1712.11
0.22 : 1765.56
0.23 : 1790.66
0.24 : 1812.84
0.25 : 1842.03
0.26 : 1893.24
0.27 : 1929.3
0.28 : 1955.4
0.29 : 1972.67
0.3 : 1993.58
0.31 : 2010.

In [15]:
cleanNetworks = '/SyntheticNetworks'
networks = next(walk(cleanNetworks), (None, None, []))[2]
networks

['networkovermu035.gml',
 'networkovermu030.gml',
 'networkovermu015.gml',
 'networkovermu010.gml',
 'networkovermu040.gml',
 'networkovermu005.gml',
 'networkovermu070.gml',
 'networkovermu025.gml',
 'networkovermu020.gml']